In [30]:
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [31]:
apple_stock = pd.read_csv("./AppleCompanyStock.csv", sep=",")

In [8]:
apple_stock.head()

,year,month,open,high,low,close,volume
0,2018,3,177.3,177.5,173.7,175.8,46812368
1,2018,3,180.3,183.5,177.6,178.0,151720299
2,2018,3,175.2,180.0,174.3,180.0,138442235
3,2018,3,176.3,180.6,172.4,176.2,196432091
4,2018,2,172.1,175.7,171.0,175.5,133197874


In [32]:
apple_stock.describe()

,year,month,open,high,low,close,volume
count,950.0,950.0,950.0,950.0,950.0,950.0,950.0
mean,2008.6,6.5,169.0,174.2,164.2,169.4,109291496.2
std,5.3,3.5,167.3,171.6,163.7,167.5,81443248.2
min,2000.0,1.0,13.1,13.6,12.7,13.1,5515100.0
25%,2004.0,3.0,46.2,48.3,44.8,46.1,43471950.0
50%,2009.0,6.5,111.4,114.7,108.2,111.9,92348400.0
75%,2013.0,9.8,195.6,202.5,190.3,195.2,151118018.0
max,2018.0,12.0,699.4,705.1,693.6,700.1,500362965.0


In [18]:
import statistics 
from statistics import median
apple_stock['sum'] = apple_stock[apple_stock.columns].sum(axis=1)
apple_stock['median'] = apple_stock[apple_stock.columns].median(axis=1)
apple_stock

,year,month,open,high,low,close,volume,sum,median
0,2018,3,177.3,177.5,173.7,175.8,46812368,93630186.5,177.4
1,2018,3,180.3,183.5,177.6,178.0,151720299,303446078.9,181.9
2,2018,3,175.2,180.0,174.3,180.0,138442235,276889930.9,180.0
3,2018,3,176.3,180.6,172.4,176.2,196432091,392869635.2,178.5
4,2018,2,172.1,175.7,171.0,175.5,133197874,266401176.4,175.6
...,...,...,...,...,...,...,...,...,...
945,2000,2,108.0,117.1,105.9,108.8,14859900,29724683.6,112.9
946,2000,2,101.0,110.0,94.5,108.0,21289500,42583831.0,109.0
947,2000,1,108.4,114.2,100.6,101.6,18470300,36945451.7,111.3
948,2000,1,101.0,121.5,100.4,111.3,30213200,60431270.5,116.4


In [19]:
my_feature = apple_stock["sum"]

feature_columns = [tf.feature_column.numeric_column("sum")]

In [20]:
targets = apple_stock["median"]

In [21]:
# Use gradient descent as the optimizer for training the model.
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

In [22]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels